# Import Libraries and Reading the Input Video

In [1]:
import cv2
import numpy as np
import imutils
# Reading video file from folder, video need to put in same folder with code
cap = cv2.VideoCapture('IMG_8096.MOV')

# Apply all the necessary techniques to detect the people in the video

In [2]:
ret,frame1=cap.read()
ret,frame2=cap.read()
while cap.isOpened():
    # If video is not finished, then proceed
    if ret:
        # Resize the previous frame and the current frame into the width of 600
        frame1 = imutils.resize(frame1, width=600)
        frame2 = imutils.resize(frame2, width=600)
        # Apply Image Differences to find the differences between previous and current frame
        diff=cv2.absdiff(frame1,frame2)
        # Convert the frame into grayscale
        gray=cv2.cvtColor(diff,cv2.COLOR_BGR2GRAY)
        # Apply Bilateral Filter to remove the noise
        blurred = cv2.bilateralFilter(gray,10,40,40)
        # Apply Canny Edge Detector to obtain the all the edges
        edged = cv2.Canny(blurred,30,200)
        # Apply Adaptive Thresholding to segment the frames
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        # Apply Bitwise Not to obtain the mask of the frame
        mask = cv2.bitwise_not(edged)
        # Apply Bitwise And to get the sharpen edges of the frame
        thresh = cv2.bitwise_and(thresh,thresh,mask=mask)
        # Apply Opening to open the connections of the people detected in the frame
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(37,37))
        thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel)
        # Apply Bitwise Not again to convert all the background frames and people detected from white to black and black to white respectively
        thresh = cv2.bitwise_not(thresh)
        # Display the result of the thresholded video in black and white
        cv2.imshow("Frame1",thresh)
        # Find Contours in the thresholded frame
        contours,_=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # Initialize counter into zero
        count=0
        for contour in contours:
            # Save all the coordinates
            (x,y,w,h)=cv2.boundingRect(contour)
            # if the area of contour is less than 5000, do nothing
            if cv2.contourArea(contour)<5000:
                continue
            # If the people are detected in the frame, the counter is increased by 1
            count=count+1
            # Draw the rectangle bounding box for each people detected in the frame
            cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
            # Display text about the total number of people detected in the lower left corner
            cv2.putText(frame1,"{} people is approaching you".format(str(count)),(10,325),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
       # Display the output video for the user
        cv2.imshow("feed",frame1)
        frame1=frame2
        ret,frame2=cap.read()
        # Press 'Q' if want to terminate the program before video end
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    # if video finished, then break
    else:
        break
cap.release()
cv2.destroyAllWindows()